# Query Blockchain Data
Python notebook with simple functions to query blockchain data via APIs from leading web3 data providers.

* [Allium](https://www.allium.so/)
* [BigQuery](https://console.cloud.google.com/marketplace/browse)
* [Chainbase](https://chainbase.online/)
* [Coherent](https://coherent.xyz/)
* [Credmark](https://credmark.com/)
* [Dune](https://dune.com/home)
* [Flipside](https://flipsidecrypto.xyz/)
* [Footprint](https://footprint.network/)
* [Spice](https://spice.xyz/)
* [Token Flow](https://tokenflow.live/)
* [Transpose](https://www.transpose.io/)

In [2]:
import json
import pandas as pd
import os
import json
from shroomdk import ShroomDK
from google.cloud import bigquery, bigquery_storage, storage
import time
import requests
import math
from spicepy import Client
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from transpose import Transpose
import datetime

In [3]:
# Read in API Keys from a separate file
with open("keys.json", "r") as keys_file:
    keys = json.load(keys_file)
    flipside_key = keys["flipside_key"]
    chainbase_key = keys["chainbase_key"]
    makerdojo_key = keys["makerdojo_key"]
    spice_key = keys["spice_key"]
    footprint_key = keys["footprint_key"]
    transpose_key = keys["transpose_key"]
    snowflake_account = keys["snowflake_account"]
    snowflake_user = keys["snowflake_user"]
    snowflake_password = keys["snowflake_password"]
    snowflake_warehouse = keys["snowflake_warehouse"]
    dune_key = keys["dune_key"]

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keys.json"

## [Flipside](https://flipsidecrypto.xyz/)

In [4]:
# Query Flipside using their Python SDK
def query_flipside(q):
    sdk = ShroomDK(flipside_key)
    result_list = []
    for i in range(1, 11):  # max is a million rows @ 100k per page
        data = sdk.query(q, page_size=100000, page_number=i)
        if data.run_stats.record_count == 0:
            break
        else:
            result_list.append(data.records)
    result_df = pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df = pd.json_normalize(each_list)
        else:
            try:
                result_df = pd.concat([result_df, pd.json_normalize(each_list)])
            except:
                continue
    result_df.drop(columns=["__row_index"], inplace=True)
    return result_df

In [5]:
q = """
select *
from ethereum.core.fact_decoded_event_logs
limit 5
"""
flipside_df = query_flipside(q)
flipside_df.head()

,block_number,block_timestamp,tx_hash,event_index,contract_address,event_name,decoded_log.from,decoded_log.to,decoded_log.value,full_decoded_log.address,full_decoded_log.data,full_decoded_log.decoded,full_decoded_log.name,decoded_log.dst,decoded_log.src,decoded_log.wad
0,11116547,2020-10-24T03:18:51.000Z,0xb5fd1c0de65fda0288441915b394cf347f1d27b989df...,126,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,Transfer,0x088ee5007c98a9677165d78dd2109ae4a3d04d0c,0x728bbe9bbee3af78ad611315076621865950b344,9202275127490350319,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,"[{'decoded': True, 'name': 'from', 'type': 'ad...",True,Transfer,NaN,NaN,NaN
1,11118178,2020-10-24T09:08:20.000Z,0x3401f069fc1326349f88c74c3170709ed7b312e16bcb...,71,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,Transfer,0x7b2da0d2b810747c49655be6443621f42c9c7e51,0xf0cb59157ae885ba8d70eb018ffc4ecc4d44b1c5,10293640000000000,0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2,"[{'decoded': True, 'name': 'from', 'type': 'ad...",True,Transfer,NaN,NaN,NaN
2,11117446,2020-10-24T06:26:54.000Z,0xb7f608ba2105e882701ee72c269d825d190ae4525848...,41,0xdac17f958d2ee523a2206206994597c13d831ec7,Transfer,0x0a98fb70939162725ae66e626fe4b52cff62c2e5,0x9cea7ffa98394e2eacccfec3d263d80751a98a17,10000000,0xdAC17F958D2ee523a2206206994597C13D831ec7,"[{'decoded': True, 'name': 'from', 'type': 'ad...",True,Transfer,NaN,NaN,NaN
3,11118190,2020-10-24T09:10:39.000Z,0xfa6fa7ef5671473e0aa357dbf577d520ee8bdbacb400...,101,0xdac17f958d2ee523a2206206994597c13d831ec7,Transfer,0x1fec7a446efcd517009b9ad2f50befc6e148b31e,0x0b35abe7d2bc51b8fe7663e019f646c411bc7563,850000000,0xdAC17F958D2ee523a2206206994597C13D831ec7,"[{'decoded': True, 'name': 'from', 'type': 'ad...",True,Transfer,NaN,NaN,NaN
4,11120986,2020-10-24T19:29:09.000Z,0x851059d73b86f1b7bca1e9520bbe133a6f32543929f5...,180,0x6b175474e89094c44da98b954eedeac495271d0f,Transfer,NaN,NaN,NaN,0x6B175474E89094C44Da98b954EedeAC495271d0F,"[{'decoded': True, 'name': 'src', 'type': 'add...",True,Transfer,0x5d3a536e4d6dbd6114cc1ead35777bab948e3643,0xebfb47a7ad0fd6e57323c8a42b2e5a6a4f68fc1a,10000000000000000000


## [Dune](https://dune.com/home)

In [6]:
# Query Dune Analytics using API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={dune_key}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [7]:
# https://dune.com/queries/2638781
q = "2638781"
dune_df = query_dune(q)
dune_df.head()

,block_date,block_number,block_time,contract_address,contract_name,event_name,index,namespace,signature,tx_hash
0,2021-10-24,13480201,2021-10-24 12:29:15.000 UTC,0x6dae8a922b66225de1728bd9e5f2b7a4bdd4699b,Souls,ApprovalForAll,454,paintingsofforgottensouls,0x17307eab39ab6107e8899845ad3d59bd9653f200f220...,0x010e68632472746fde74e715e011ade31e90e0e63f40...
1,2021-10-24,13480231,2021-10-24 12:35:04.000 UTC,0x6dae8a922b66225de1728bd9e5f2b7a4bdd4699b,Souls,Transfer,229,paintingsofforgottensouls,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x2583ee5b3592164854d862913846a2fa3abbbc5c51f5...
2,2021-10-24,13480231,2021-10-24 12:35:04.000 UTC,0x6dae8a922b66225de1728bd9e5f2b7a4bdd4699b,Souls,Approval,228,paintingsofforgottensouls,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...,0x2583ee5b3592164854d862913846a2fa3abbbc5c51f5...
3,2022-08-14,15336568,2022-08-14 00:25:46.000 UTC,0x6dae8a922b66225de1728bd9e5f2b7a4bdd4699b,Souls,ApprovalForAll,454,paintingsofforgottensouls,0x17307eab39ab6107e8899845ad3d59bd9653f200f220...,0x99dc9f82635ba246ac2ab6914a4004aac437d3ea5dcb...
4,2021-09-21,13269425,2021-09-21 13:33:35.000 UTC,0x6dae8a922b66225de1728bd9e5f2b7a4bdd4699b,Souls,Transfer,60,paintingsofforgottensouls,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xccb19e56d0ed7f4b73c22a01014122dd416651b780a4...


## [Google BigQuery](https://console.cloud.google.com/marketplace/browse)

In [8]:
# Query BigQuery using their Python SDK
def query_bigquery(q):
    connection = bigquery_storage.BigQueryReadClient()
    bqclient = bigquery.Client()
    results_df = bqclient.query(q).result().to_dataframe(bqstorage_client=connection)
    return results_df

In [9]:
q = """
SELECT *
FROM `bigquery-public-data.crypto_ethereum.logs` 
WHERE DATE(block_timestamp) = "2023-06-15" 
LIMIT 5
"""
bigquery_df = query_bigquery(q)
bigquery_df.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash
0,40,0xec36abf9fefdb49d13e577fabf17661d34c3461ae3cf...,11,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0xffffffffffffffffffffffffffffffffffffffffffff...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2023-06-15 13:22:11+00:00,17485546,0xa2ffad8509977b490abc3e73d2514f737b3d83cd9e73...
1,79,0x8d7ce2cd6a78269bd40ebf237632c9c359089e7bb099...,36,0xdac17f958d2ee523a2206206994597c13d831ec7,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2023-06-15 13:22:11+00:00,17485546,0xa2ffad8509977b490abc3e73d2514f737b3d83cd9e73...
2,10,0x311657a2704fa773c53afb5cfcdbf13b706bd90963d1...,1,0x2de9f12a2a01d5c55d4b2be8111f9f51143d980a,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2023-06-15 13:22:23+00:00,17485547,0xc1949a55d8c8ffb7ab13344299aa9aca29ea58ccaf81...
3,151,0x83012d29460181ac91c6b63102bbfd1a0cba5768c883...,85,0x3b3ae790df4f312e745d270119c6052904fb6790,0x000000000000000000000000eeeeeeeeeeeeeeeeeeee...,[0x1bb43f2da90e35f7b0cf38521ca95a49e68eb42fac4...,2023-06-15 13:22:23+00:00,17485547,0xc1949a55d8c8ffb7ab13344299aa9aca29ea58ccaf81...
4,59,0x1047c06533ec744a9eb4554b8aad685107be54c1e3e0...,14,0xdac17f958d2ee523a2206206994597c13d831ec7,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2023-06-15 13:22:23+00:00,17485547,0xc1949a55d8c8ffb7ab13344299aa9aca29ea58ccaf81...


## [Transpose](https://www.transpose.io/)

In [10]:
# Query Transpose using their API
def query_transpose(q):
    url = "https://api.transpose.io/sql"
    headers = {
        "Content-Type": "application/json",
        "X-API-KEY": transpose_key,
    }
    response = requests.post(
        url,
        headers=headers,
        json={
            "sql": q,
        },
    )
    results_json = json.loads(response.text)["results"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [11]:
q = f"""
select *
from ethereum.logs
limit 5
"""
transpose_df = query_transpose(q)
transpose_df.head()

,transaction_hash,log_index,address,topic_0,topic_1,topic_2,topic_3,data,timestamp,block_number,transaction_position,__confirmed
0,0x6086c4b306f88843d0a487262c5d1758995a404e2aa3...,13,0xb89570f6AD742CB1fd440A930D6c2A2eA29c51eE,0x276856b36cbc45526a0ba64f44611557a2a8b68662c5...,0x000000000000000000000000c0829421c1d260bd3cb3...,0x0000000000000000000000001f573d6fb3f13d689ff8...,0x000000000000000000000000f20b9e713a33f61fa387...,0x00000000000000000000000000000000000000000000...,2019-01-18T09:39:43Z,7086461,3,True
1,0x6086c4b306f88843d0a487262c5d1758995a404e2aa3...,14,0xb89570f6AD742CB1fd440A930D6c2A2eA29c51eE,0x8a6a7f53b3c8fa1dc4b83e3f1be668c1b251ff8d44cd...,0x000000000000000000000000c0829421c1d260bd3cb3...,None,None,0x0000000000000000000000000000000000000000003e...,2019-01-18T09:39:43Z,7086461,3,True
2,0x6086c4b306f88843d0a487262c5d1758995a404e2aa3...,15,0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...,0x000000000000000000000000f20b9e713a33f61fa387...,0x0000000000000000000000008fd5bfbc2f61a450400a...,None,0x00000000000000000000000000000000000000000000...,2019-01-18T09:39:43Z,7086461,3,True
3,0x8fa9324b721a4997585f2ea029f9a02ed719e1260707...,5,0x77a4ab25422342c6471d380644FD3A27d9b3752D,0xf808965353239bb28ad831b3c6efd69ed60fab95008c...,None,None,None,0x00000000000000000000000000000000000000000000...,2019-01-18T16:48:43Z,7088103,9,True
4,0xc18b2e1ed0dfec4e7d37b533d76b45254271355728ea...,6,0x7064aAb39A0Fcf7221c3396719D0917a65E35515,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x0000000000000000000000003cb642f68b1e1b5cf7cf...,0x000000000000000000000000ae028ab358529b0cf775...,None,0x00000000000000000000000000000000000000000018...,2019-01-18T16:48:43Z,7088103,10,True


## [Chainbase](https://chainbase.online/)

In [13]:
# Query Chainbase using their REST API
def query_chainbase(q):
    url = "https://api.chainbase.online/v1/dw/query"
    payload = {"query": q}
    headers = {"x-api-key": chainbase_key}
    response = requests.post(url, json=payload, headers=headers)
    task_id = response.json()["data"]["task_id"]
    row_ct = response.json()["data"]["rows"]
    all_results_df = pd.DataFrame([])
    row_ct = math.ceil(response.json()["data"]["rows"] / 1000)
    if row_ct == 1:
        data = response.json()["data"]["result"]
        all_results_df = pd.json_normalize(data)
    else:
        time.sleep(2)
        for i in range(0, row_ct, 1):
            payload = {"task_id": task_id, "page": i + 1}
            response = requests.post(url, json=payload, headers=headers)
            data = response.json()["data"]["result"]
            results_df = pd.json_normalize(data)
            all_results_df = pd.concat([all_results_df, results_df])
            time.sleep(2)
    return all_results_df

In [23]:
q = """
select *
from ethereum.transaction_logs
limit 5
"""
chainbase_df = query_chainbase(q)
chainbase_df.head()

,address,block_number,block_timestamp,data,log_index,topic0,topic1,topic2,topic3,topics_count,transaction_hash,transaction_index
0,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,5225152,2018-03-09 15:45:43,0x00000000000000000000000000000000000000000000...,21,0xf341246adaac6f497bc2a656f546ab9e182111d63039...,,,,1,0x000b8ee03444b83146be8c04fad3a65298e6910a60f1...,41
1,0x8713d26637cf49e1b6b4a7ce57106aabc9325343,5225070,2018-03-09 15:26:41,0x00000000000000000000000000000000000000000000...,96,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x00000000000000000000000079b76cf32eee48b4b335...,0x0000000000000000000000006caf22e576b80954b5a8...,,3,0x000c4f6009bdb94a8651d83c344513a83e1e303df360...,69
2,0xc5d105e63711398af9bbff092d4b6769c82f793d,5225128,2018-03-09 15:39:26,0x00000000000000000000000000000000000000000000...,111,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x000000000000000000000000f8385c418cb51ce843b7...,0x000000000000000000000000bcc6d7b2e5a1cadcfef5...,,3,0x000ca48054d5e22ef75d947ad265365113cf79599fb2...,43
3,0xc5d105e63711398af9bbff092d4b6769c82f793d,5225128,2018-03-09 15:39:26,0x00000000000000000000000000000000000000000000...,112,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x000000000000000000000000f8385c418cb51ce843b7...,0x0000000000000000000000002b52d800d91d62b5defe...,,3,0x000ca48054d5e22ef75d947ad265365113cf79599fb2...,43
4,0xc5d105e63711398af9bbff092d4b6769c82f793d,5225128,2018-03-09 15:39:26,0x00000000000000000000000000000000000000000000...,113,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x000000000000000000000000f8385c418cb51ce843b7...,0x000000000000000000000000713d7e35455b211e3228...,,3,0x000ca48054d5e22ef75d947ad265365113cf79599fb2...,43


## [Spice](https://spice.xyz/)

In [15]:
# Query Spice using their SDK
def query_spice(q):
    client = Client(spice_key)
    reader = client.query(q)
    results_df = reader.read_pandas()
    return results_df

In [24]:
q = """
select *
from eth.logs
limit 5
"""
spice_df = query_spice(q)
spice_df.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_hash,block_number
0,238,0x2dc106a2cbbc4560e7c0b6c361010b55f158e9ae4d66...,137,0x36b73d268a441164ea0e4e5560a579573a475ebf,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,1686830759,0xdc1a1f7c94422e104f8961aa8d60d340184ef09dc9d1...,17485170
1,239,0x2dc106a2cbbc4560e7c0b6c361010b55f158e9ae4d66...,137,0xd1d1d4e36117ab794ec5d4c78cbd3a8904e691d0,0x00000000000000000000000000000000000000000000...,[0x3d0ce9bfc3ed7d6862dbb28b2dea94561fe714a1b4d...,1686830759,0xdc1a1f7c94422e104f8961aa8d60d340184ef09dc9d1...,17485170
2,240,0x2dc106a2cbbc4560e7c0b6c361010b55f158e9ae4d66...,137,0x36b73d268a441164ea0e4e5560a579573a475ebf,0x00000000000000000000000000000000000000000000...,[0x6f8da53cfedb8cc4f7935c3629624e50b63053c93bb...,1686830759,0xdc1a1f7c94422e104f8961aa8d60d340184ef09dc9d1...,17485170
3,241,0x2dc106a2cbbc4560e7c0b6c361010b55f158e9ae4d66...,137,0x36b73d268a441164ea0e4e5560a579573a475ebf,0x00000000000000000000000000000000000000000000...,[0x4e26b0356a15833a75d497ecc40ebbb716b99466ed0...,1686830759,0xdc1a1f7c94422e104f8961aa8d60d340184ef09dc9d1...,17485170
4,65,0x15502cba6fc30ef2c3fd5f7d2feb33ed422a50e91dc9...,48,0xdac17f958d2ee523a2206206994597c13d831ec7,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,1686830759,0xdc1a1f7c94422e104f8961aa8d60d340184ef09dc9d1...,17485170


## [Coherent](https://coherent.xyz/)

In [18]:
# Query Coherent using their Snowflake API
def query_snowflake(q):
    conn = create_engine(
        URL(
            account=snowflake_account,
            user=snowflake_user,
            password=snowflake_password,
            warehouse=snowflake_warehouse,
        )
    )
    connection = conn.raw_connection()
    q_results = connection.cursor().execute(q)
    result_df = q_results.fetch_pandas_all()
    result_df.columns = result_df.columns.str.lower()
    return result_df

In [19]:
q = """
select *
from ethereum_dataset_raw_decoded_and_enriched.decoded.decoded_logs
limit 5
"""
coherent_df = query_snowflake(q)
coherent_df.head()

,address,block_hash,block_number,data,log_index,topics,transaction_hash,transaction_index,removed,hashable_signature,decoded_result
0,0xdac17f958d2ee523a2206206994597c13d831ec7,0xc39fb8952b777f65881c60ff9dff8e3699da08db4d7c...,17316459.0,0x00000000000000000000000000000000000000000000...,7.0,"[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163...",0x3992cacb8189fa5aee10813bcd6902a61398f54061bf...,12.0,False,"Transfer(address,address,uint256)","{\n ""from"": ""0x1bdae82b6d4be856897eedcbcfcbd7..."
1,0x8bf5e55e943ca10f7a67c92c670bb9a2cc5f02a4,0x153471a029e2610a5c7928e2878ada8c71e3a9c7803e...,17316430.0,0x00000000000000000000000000000000000000000000...,126.0,"[""0xd78ad95fa46c994b6551d0da85fc275fe613ce3765...",0x001cf129288bfd02a5e18b687e26a6fc5de6ffa33ab5...,45.0,False,"Swap(address,uint256,uint256,uint256,uint256,a...","{\n ""amount0In"": ""0"",\n ""amount0Out"": 1.0417..."
2,0x069fd302ee389da48acb84a97b3ec15092f8c9b8,0x153471a029e2610a5c7928e2878ada8c71e3a9c7803e...,17316430.0,0x000000000000000000000000000000000000000d25f9...,124.0,"[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163...",0x001cf129288bfd02a5e18b687e26a6fc5de6ffa33ab5...,45.0,False,"Transfer(address,address,uint256)","{\n ""from"": ""0x8bf5e55e943ca10f7a67c92c670bb9..."
3,0x1d4e66b826c8bf8e3c93574386a8dd723b3a912f,0xee59142cb1ebbfd845c9c8d5b205298f21d1426f0749...,17316421.0,0x00000000000000000000000000000000000000000fa5...,235.0,"[""0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c7...",0x03f1307ef4991fca7c956a6e69d2b2ef7f842e4626fa...,102.0,False,"Sync(uint112,uint112)","{\n ""reserve0"": 4.841955299776536e+27,\n ""re..."
4,0x069fd302ee389da48acb84a97b3ec15092f8c9b8,0x7a927fce71b9c37053376d19442c82eaba992b848bbe...,17316415.0,0x000000000000000000000000000000000000000d4313...,401.0,"[""0xddf252ad1be2c89b69c2b068fc378daa952ba7f163...",0x18719271b0c6b5ede9e9593eed5e6e97da74e388b80d...,205.0,False,"Transfer(address,address,uint256)","{\n ""from"": ""0x8bf5e55e943ca10f7a67c92c670bb9..."


## [Allium](https://www.allium.so/)

In [25]:
q = """
select *
from blockchain_data_ethereum_raw__assets.assets.balances -- allium
limit 5
"""
allium_df = query_snowflake(q)
allium_df.head()

,address,token_address,token_name,token_symbol,token_id,raw_balance,raw_balance_str,balance,balance_str,usd_balance,usd_exchange_rate,block_timestamp,block_number,block_hash,token_type,unique_id
0,0x1c813ada3a7a65345c09a0c836a0bcca4fe46203,0x0000000000000000000000000000000000000000,ETH,ETH,None,5.580000e+15,5.58E+15,0.005580,0.00558,0.497122,89.09,2018-12-08 11:42:51,6848426,0x3d7266b1a3d5c0e3b2261c3e586c21010f0201576e21...,ETH,block-6848426_address-0x1c813ada3a7a65345c09a0...
1,0xc2b8a97c05e177aeea9a31a1f108f13d5244f46d,0x0000000000000000000000000000000000000000,ETH,ETH,None,1.893650e+15,1.89365E+15,0.001894,0.00189365,0.165884,87.60,2018-12-08 15:11:18,6849267,0x8506307c10cb1239349f42343005880b005ff556cf07...,ETH,block-6849267_address-0xc2b8a97c05e177aeea9a31...
2,0x6dbff98909acf14533571769ede1298f353e2216,0x0000000000000000000000000000000000000000,ETH,ETH,None,6.157909e+17,6.15790919E+17,0.615791,0.615790919,56.788239,92.22,2018-12-08 06:41:47,6847171,0xaec8c59f79dba2ad85fc1b52aaaef3ea3d8bccad7bae...,ETH,block-6847171_address-0x6dbff98909acf145335717...
3,0xc9d60bff572994049821c1f93bb81e4f05b9fb4d,0x0000000000000000000000000000000000000000,ETH,ETH,None,1.373886e+16,1.373885702E+16,0.013739,0.01373885702,1.269333,92.39,2018-12-08 23:56:56,6851502,0x919abdb8fadb92d3e87fe23a0703749fcd8d86699073...,ETH,block-6851502_address-0xc9d60bff572994049821c1...
4,0x11597f68a6302417e51aaa28d4b9999ae323e3eb,0x0000000000000000000000000000000000000000,ETH,ETH,None,2.030022e+15,2.0300218E+15,0.002030,0.0020300218,0.179251,88.30,2018-12-08 13:26:37,6848850,0xd56d4ad795987240656b9591cd18fd5526eaa54a8ad0...,ETH,block-6848850_address-0x11597f68a6302417e51aaa...


## [Credmark](https://credmark.com/)

In [20]:
q = """
select *
from ethereum__decoded__core.decoded.events
limit 5
"""
credmark_df = query_snowflake(q)
credmark_df

,transaction_hash,log_index,contract_address,block_number,decoded,name,signature,block_timestamp
0,0xa16c5145d5125ccd302fdd99962e1ee9315597b9687c...,192,0x6b175474e89094c44da98b954eedeac495271d0f,15764287,"{""name"": ""Transfer"", ""signature"": ""Transfer(ad...",Transfer,"Transfer(address,address,uint256)",2022-10-17 00:35:11
1,0xa16c5145d5125ccd302fdd99962e1ee9315597b9687c...,196,0x6b175474e89094c44da98b954eedeac495271d0f,15764287,"{""name"": ""Transfer"", ""signature"": ""Transfer(ad...",Transfer,"Transfer(address,address,uint256)",2022-10-17 00:35:11
2,0xc5994b8386205ca2fb3acf86ecee4aa0745bb16e618b...,14,0x6b175474e89094c44da98b954eedeac495271d0f,15764288,"{""name"": ""Transfer"", ""signature"": ""Transfer(ad...",Transfer,"Transfer(address,address,uint256)",2022-10-17 00:35:23
3,0x6b45130e2ce1a6bef7e82161d0ff40ba581293e4b78f...,266,0x6b175474e89094c44da98b954eedeac495271d0f,15764290,"{""name"": ""Transfer"", ""signature"": ""Transfer(ad...",Transfer,"Transfer(address,address,uint256)",2022-10-17 00:35:47
4,0x9292a01a04ce29981b4245c0c89f14cc610cf199aa66...,100,0x6b175474e89094c44da98b954eedeac495271d0f,15764294,"{""name"": ""Approval"", ""signature"": ""Approval(ad...",Approval,"Approval(address,address,uint256)",2022-10-17 00:36:35


## [Token Flow](https://tokenflow.live/)

In [21]:
q = """
select *
from ethereum_data_warehouse_beta.decoded_sample.events
limit 5
"""
tokenflow_df = query_snowflake(q)
tokenflow_df

,load_id,chain_id,block,timestamp,tx_hash,log_index,call_id,contract_address,contract_name,event_name,event_signature,raw_parameters,parameters,status,order_index,decoding_status
0,2022-05-05 17:35:18,mainnet,14606148,2022-04-18 01:06:43,0x1d887e4910b21727332e8730e4952f841eb6ee10eee1...,187,0_4,0x1a9e22b930daa5226603b9c99974055a03c655a1,UniswapV2Pair,Swap,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,"[\n {\n ""name"": ""sender"",\n ""raw"": ""0x0...","{\n ""amount0In"": 10909744203431284498777671,\...",True,2872,True
1,2022-05-05 17:35:18,mainnet,14606148,2022-04-18 01:06:43,0x1d887e4910b21727332e8730e4952f841eb6ee10eee1...,188,1_1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Withdrawal,0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98c...,"[\n {\n ""name"": ""src"",\n ""raw"": ""0x0000...","{\n ""src"": ""0x68b3465833fb72a70ecdf485e0e4c7b...",True,2886,True
2,2022-05-05 17:35:18,mainnet,14606148,2022-04-18 01:06:43,0x1d887e4910b21727332e8730e4952f841eb6ee10eee1...,185,0_4_0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Transfer,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,"[\n {\n ""name"": ""src"",\n ""raw"": ""0x0000...","{\n ""dst"": ""0x68b3465833fb72a70ecdf485e0e4c7b...",True,2859,True
3,2022-05-05 17:35:18,mainnet,14606148,2022-04-18 01:06:43,0xb79a4597f1e94d1c8726346797f78cad3ac5dc459f8f...,189,None,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Transfer,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,"[\n {\n ""name"": ""from"",\n ""raw"": ""0x000...","{\n ""from"": ""0xddfabcdc4d8ffc6d5beaf154f18b77...",True,2900,True
4,2022-05-05 17:35:18,mainnet,14606148,2022-04-18 01:06:43,0xec4adb08ac174bd247660ed897181caef7f89c17cc0e...,190,None,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Transfer,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,"[\n {\n ""name"": ""from"",\n ""raw"": ""0x000...","{\n ""from"": ""0x21a31ee1afc51d94c2efccaa2092ad...",True,2915,True


## [Footprint](https://footprint.network/)

In [22]:
# Query Footprint using their API
def query_footprint(q):
    url = "https://api.footprint.network/api/v1/native"

    payload = {"query": q}
    headers = {
        "accept": "application/json",
        "API-KEY": footprint_key,
        "content-type": "application/json",
    }

    response = requests.post(url, json=payload, headers=headers)
    results_json = json.loads(response.text)["data"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
q = """
select *
from ethereum_blocks
"""
footprint_df = query_footprint(q)
footprint_df